# Quench dynamics in the MBL phase

$\require{physics}$

In [ ]:
from copy import deepcopy

import numpy as np
import pandas as pd
from scipy.linalg import expm
import matplotlib.pyplot as plt
%matplotlib inline

from ph121c_lxvm import tensor, measure

## Summary

We will again evolve this quantum state
\begin{align}
    \ket{\psi(t=0)} = \ket{\xi} \otimes \cdots \otimes \ket{\xi}
    ,
\end{align}
where $\ket{\xi} = \frac{1}{2} \left( \ket{\uparrow} - \sqrt{3} \ket{\downarrow} \right)$.

In [ ]:
# Build initial state
down = np.array([1., 0.]).reshape(2, 1)
up   = down[::-1].reshape((2, 1))
ξ = (up - np.sqrt(3) * down) / 2

def make_wave (L, d):
    """Create the inital wavefunctions."""
    ψ = tensor.mps(L=L, d=d)
    ψ.from_arr([ ξ for _ in range(L) ], center=-1)  
    return ψ

Measuring the half-system entanglement entropy:

In [ ]:
def eentropy (tren, i):
    """Calculate the entanglement entropy at a cut between quanta i, i+1."""
    tren.groupby_quanta_tag([i])
    u, s, vh = np.linalg.svd(tren.center.mat)
    return measure.entropy.entanglement(s)

We will use the following TFIM Hamiltonian with open boundary conditions
parametrized by random coefficients $h_i^x, h_i^z \sim U(-W, W)$:
$$
    H = 
        -\sum_{i=1}^{L-1} \sigma_i^z \sigma_{i+1}^z
        -\sum_{i=1}^L h_i^x \sigma_i^x 
        -\sum_{i=1}^L h_i^z \sigma_i^z 
    .
$$

In [ ]:
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.diag([1, -1])

# Build pieces of Hamiltonian in gate representation
def build_pieces_of_H (L, d, W, seed):
    """Build the field, odd, and even term Hamiltonians and also their union."""
    rng = np.random.default_rng(seed=seed)
    H_field = np.empty(L, dtype='O')
    for i in range(H_field.size):
        H_field[i] = tensor.mpo(L=L, d=d)
        H_field[i].set_local_oper(
            -(rng.random((-W, W)) * sx + rng.random((-W, W)) * sz), i + 1
        )
    H_odd = np.empty(L//2, dtype='O')
    for i in range(H_odd.size):
        H_odd[i] = tensor.mpo(L=L, d=d)
        H_odd[i].set_local_oper(-sz, 2*i + 1)
        H_odd[i].set_local_oper(sz, 2*i + 1 + 1)
    H_even = np.empty(L//2 - 1 + L%2, dtype='O')
    for i in range(H_even.size):
        H_even[i] = tensor.mpo(L=L, d=d)
        H_even[i].set_local_oper(-sz, 2*(i + 1))
        H_even[i].set_local_oper(sz, 2*(i + 1) + 1)
    H_full = np.array([*H_field, *H_odd, *H_even], dtype='O')
    return (H_field, H_odd, H_even, H_full)


# Construct propagators
def build_propagators (L, d, δτ, H_field, H_odd, H_even):
    """Exponentiate each non-commuting piece of the Hamiltonian"""
    U_field = tensor.mpo(L=L, d=d)
    for i, e in enumerate(H_field):
        U_field.set_local_oper(expm(- 1.j * δτ * e[0].mat), i+1)
    U_odd = tensor.mpo(L=L, d=d)
    for i, e in enumerate(H_odd):
        U_odd.set_local_oper(
            expm(- 1.j * δτ * np.kron(e[1].mat, e[0].mat)),
            2 * i + 1
        )
    U_even = tensor.mpo(L=L, d=d)
    for i, e in enumerate(H_even):
        U_even.set_local_oper(
            expm(- 1.j * δτ * np.kron(e[1].mat, e[0].mat)),
            2 * (i + 1)
        )
    return (U_field, U_odd, U_even)



## Evolution

We will evolve the state in real time and measure the entanglement entropy
at each time step.
The goal is to observe the MBL physics of this Hamiltonian through the
logarithmic growth of the entanglement entropy.

In [ ]:
W = 3
L = 64
d = 2
hx = 1.05
hz = 0.5
δτ = 0.1
N = 50
seeds = [935, 483, 102, 567, 304]

In [ ]:
ψ = make_wave(L=L, d=d)
# Data structure to save results
ψ_results = dict(S=[], seed=[], n=[])

In [ ]:
%%time
chi = 16
Nstp = 20
for seed in seeds:
    H_field, H_odd, H_even, H_full = build_pieces_of_H(
        L=L, d=d, W=W, seed=seed
    )
    U_field, U_odd, U_even = build_propagators(
        L=L, d=d, δτ=δτ, H_field=H_field, H_odd=H_odd, H_even=H_even
    )
    wave = deepcopy(ψ)
    # TEBD pattern
    for i in range(Nstp):
        for e in [U_field, U_even, U_odd]:
            e.oper(wave, inplace=True)
        wave.canonize(center=-1)
#         wave.trim_bonds(chi)
        wave.normalize()
        # measure entropy
        ψ_results['S'].append(eentropy(wave, L//2))
        ψ_results['n'].append(i)
        ψ_results['seed'].append(seed)

## Plots

Here we will plot the entanglement entropy as a function of time for all
the disorder realizations.

In [ ]:
df = pd.DataFrame(ψ_results)

In [ ]:
# nrow = len(graphs)
# ncol = len(opers)
# fig, axes = plt.subplots(nrow, ncol)
# for i, row in enumerate(axes):
#     for j, ax in enumerate(row):
#         for k in range(Navg):
#             ax.plot(
#                 np.arange(Nstp)*dt,
#                 df[(df.Pauli == opers[j]) & (df.L == graphs[i]) & (df.i == k)
#                 ].vals.values[0], alpha=0.3
#             )
#         ax.plot(
#             np.arange(Nstp)*dt,
#             np.mean(df[(df.Pauli == opers[j]) & (df.L == graphs[i])].vals.values)
#         )
#         ax.axhline(np.mean(observe[opers[j]][i*Navg:(i+1)*Navg]))
#         ax.set_title(f"$\sigma_0^{opers[j]}, L={graphs[i]}$")
#         ax.set_xlabel('t')
#         ax.set_ylabel(f"$\\langle \sigma_0^{opers[j]} (t) \\rangle$")
# fig.set_size_inches(9, 9)
# fig.tight_layout()
# plt.show()